In [29]:
import pandas as pd
import numpy as np
import os

In [30]:
path = "C:\\Users\\pl72955\\Documents\\kaggle\\"

In [31]:
DATA_FOLDER = path

transactions    = pd.read_csv(os.path.join(DATA_FOLDER, 'sales_train_v2.csv'))
# items           = pd.read_csv(os.path.join(DATA_FOLDER, 'items.csv'))
# item_categories = pd.read_csv(os.path.join(DATA_FOLDER, 'item_categories.csv'))
# shops           = pd.read_csv(os.path.join(DATA_FOLDER, 'shops.csv'))
train = transactions

In [32]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [33]:
train['date'] = pd.to_datetime(train['date'],format="%d.%m.%Y")

In [34]:
train['month'] = train['date'].dt.month
train['year'] = train['date'].dt.year
train['monthyear'] = train['month'].map(str)+train['year'].map(str)

In [35]:
train['shopitem'] = train['shop_id'].map(str)+train['item_id'].map(str)

In [36]:
#restrict to 2015
# train = train[train.year==2015]

In [37]:
train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,month,year,monthyear,shopitem
0,2013-01-02,0,59,22154,999.00,1.0,1,2013,12013,5922154
1,2013-01-03,0,25,2552,899.00,1.0,1,2013,12013,252552
2,2013-01-05,0,25,2552,899.00,-1.0,1,2013,12013,252552
3,2013-01-06,0,25,2554,1709.05,1.0,1,2013,12013,252554
4,2013-01-15,0,25,2555,1099.00,1.0,1,2013,12013,252555


In [38]:
monthgroup = train.groupby([ 'shopitem', 'monthyear',])['item_cnt_day'].sum()
monthgroupdf = pd.DataFrame(monthgroup)

In [39]:
monthgroupdf.reset_index(inplace=True)  
monthgroupdf.head()

,shopitem,monthyear,item_cnt_day
0,01000,12013,5.0
1,01000,22013,4.0
2,010004,22013,1.0
3,01001,12013,2.0
4,010012,12013,1.0


In [40]:
mys = len(monthgroupdf.monthyear.unique()) 

In [41]:
monthgroupdf.item_cnt_day = monthgroupdf.item_cnt_day.divide(mys)

In [42]:
naive_results = pd.DataFrame(monthgroupdf.groupby('shopitem')['item_cnt_day'].sum())

In [43]:
naive_results.reset_index(inplace=True)  

In [44]:
naive_results.head()

,shopitem,item_cnt_day
0,01000,0.264706
1,010004,0.029412
2,01001,0.058824
3,010012,0.088235
4,01002,0.058824


In [45]:
test = pd.read_csv(os.path.join(DATA_FOLDER, 'test.csv'))

In [46]:
test['shopitem'] = test['shop_id'].map(str)+test['item_id'].map(str)
submit = test.merge(naive_results, on='shopitem', how='left')

In [47]:
submit['item_cnt_day'] = submit['item_cnt_day'].fillna(.25)

In [48]:
submit = submit[['ID','item_cnt_day']]

submit.item_cnt_day = submit.item_cnt_day.clip(lower=0,upper=20)
submit = submit.rename(columns={'item_cnt_day':'item_cnt_month'})

In [49]:
submit.head()

,ID,item_cnt_month
0,0,0.382353
1,1,0.250000
2,2,0.294118
3,3,0.029412
4,4,0.029412


In [50]:
submit.sort_values(by=['item_cnt_month'], ascending=False).head()

,ID,item_cnt_month
37350,37350,20.0
39758,39758,20.0
46174,46174,20.0
46989,46989,20.0
157330,157330,20.0


In [51]:
submit.describe()

,ID,item_cnt_month
count,214200.000000,214200.000000
mean,107099.500000,0.327228
std,61834.358168,0.741271
min,0.000000,0.000000
25%,53549.750000,0.117647
50%,107099.500000,0.250000
75%,160649.250000,0.250000
max,214199.000000,20.000000


In [52]:
submit.to_csv('submit3.csv' , index=False, header=True)